In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/covid19-global-forecasting-week-5/train.csv')
train['County']=train['County'].fillna("NR")
train['Province_State']=train['Province_State'].fillna("NR")
train.head()

In [ ]:
a=train.groupby(['County','Province_State','Country_Region','Target'])['TargetValue'].quantile(q=0.05).reset_index()
b=train.groupby(['County','Province_State','Country_Region','Target'])['TargetValue'].quantile(q=0.5).reset_index()
c=train.groupby(['County','Province_State','Country_Region','Target'])['TargetValue'].quantile(q=0.95).reset_index()
a.columns=['County','Province_State','Country_Region','Target','q0.05']
b.columns=['County','Province_State','Country_Region','Target','q0.5']
c.columns=['County','Province_State','Country_Region','Target','q0.95']
a=pd.concat([a,b['q0.5'],c['q0.95']],1)
a['q0.05']=a['q0.05'].clip(0,10000)
a['q0.5']=a['q0.5'].clip(0,10000)
a['q0.95']=a['q0.95'].clip(0,10000)
a.head()

In [ ]:
test = pd.read_csv('../input/covid19-global-forecasting-week-5/test.csv')
test['County']=test['County'].fillna("NR")
test['Province_State']=test['Province_State'].fillna("NR")
test.head()

In [ ]:
test=test.merge(a,on=['Country_Region','County','Province_State','Target'],how='left')
test.head()


In [ ]:
test2=test[['ForecastId','q0.05','q0.5','q0.95']]
test2.head()

In [ ]:
sub=pd.melt(test2, id_vars=['ForecastId'], value_vars=['q0.05','q0.5','q0.95'])
sub['variable']=sub['variable'].str.replace("q","", regex=False)
sub['ForecastId_Quantile']=sub['ForecastId'].astype(str)+'_'+sub['variable']
sub['TargetValue']=sub['value']
sub=sub[['ForecastId_Quantile','TargetValue']]
sub.reset_index(drop=True,inplace=True)
sub.to_csv("submission.csv",index=False)
sub.head()